In [ ]:
import duckdb
import pandas as pd
from pathlib import Path
from data.settings import settings
from data.alpha_vantage_schema import TABLE_SCHEMAS

In [ ]:
db_path = Path(settings.get("data_dir"), settings.get("db_name"))
conn = duckdb.connect(str(db_path), read_only=False)

In [ ]:
result = conn.execute("DROP TABLE FUNDAMENTALS").df()
print(result)

In [ ]:
conn.close()